In [ ]:
import ee
import geemap
import xml.etree.ElementTree as ET
import ipywidgets as widgets
import pandas as pd 
from pprint import pprint 

In [ ]:
ee.Authenticate()

In [ ]:
ee.Initialize(project="jameswilliamchamberlain")

In [ ]:
# collect Samarra Pin

# UNESCO World Heritage Sites List xml file 
tree = ET.parse("whs.xml")
root = tree.getroot()

data = []

for row in root.findall("row"):
    site = row.findtext('site')
    category = row.findtext('category')
    date_inscribed = row.findtext('date_inscribed')
    region = row.findtext('region')
    url = row.findtext('http_url')
    longitude = row.findtext('longitude')
    latitude = row.findtext('latitude')
    iso = row.findtext('iso_code')
    iso = iso.split(',')
    data.append([site, category, date_inscribed, region, url, longitude, latitude, iso])

df = pd.DataFrame(data, columns=["name", "category", "date inscribed", "region", "url", "longitude", "latitude", "iso"])

df["longitude"] = pd.to_numeric(df["longitude"], errors='coerce')
df["latitude"] = pd.to_numeric(df["latitude"], errors='coerce')

# drop empties 
df = df.dropna(subset=["longitude", "latitude"]).reset_index(drop=True) 

# filter out everything but Samarra 
target_url = 'https://whc.unesco.org/en/list/276'
df_disp = df[df['url'] == target_url]
df_disp = df_disp[df_disp['url'] == target_url]
df_disp = df_disp.reset_index(drop=True)

In [ ]:
m = geemap.Map()
m.add_points_from_xy(df_disp, x="longitude", y="latitude", layer_name="Sites")

In [ ]:
# util functions and widgets
def map_polygon(polygon, collection_name, layer_name, yyyymmdd1="2024-01-01", yyyymmdd2="2024-12-29", num_tasks=10):

    collection = ee.ImageCollection(collection_name) \
        .filterDate(yyyymmdd1, yyyymmdd2) \
        .filterBounds(polygon) \
        .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 20)) \
        .median() \
        .clip(polygon)
    
    vis = {'min': 0, 'max': 3000, 'bands': ['B4', 'B3', 'B2']}

    m.addLayer(collection, vis, layer_name)

def draw_rectangles_for_date(date_val, size_deg=1):

    regions = []

    for _, row in df_disp.iterrows():
        lat = row['latitude']
        lon = row['longitude']

        geometry = ee.Geometry.Rectangle([lon - size_deg/2, lat - size_deg/2, lon + size_deg/2, lat + size_deg/2])
        regions.append(geometry)

    regions_collection = ee.FeatureCollection(regions)

    map_polygon(regions_collection, "COPERNICUS/S2_HARMONIZED", f"S2 {date_val}", f"{date_val}-01-01", f"{date_val}-3-31")

def draw_basic(b):
    dates = [2025, 2020, 2016]
    # dates = [2025, 2024, 2022, 2020, 2018, 2016]
    # dates = [2025, 2024, 2023, 2022, 2021, 2020, 2019, 2018, 20217, 2016]

    for date in dates:
        draw_rectangles_for_date(date)

def draw_selected(b):

    if m.draw_features is None:
        raise ValueError("Error: no roi")
    
    # roi = m.draw_last_feature.geometry()
    roi = ee.FeatureCollection(m.draw_features)

    year_start_val = int(date_slider.value)
    month_start_val = int(month_slider.value)
    time_frame_val = int(time_frame_slider.value)

    month_end_val = month_start_val + time_frame_val
    year_overflow = 0
    if month_end_val > 12:
        month_end_val -= 12
        year_overflow = 1

    yyyymm_start = f"{year_start_val}-{month_start_val}"
    yyyymm_end = f"{year_start_val + year_overflow}-{month_end_val}"
    
    map_polygon(roi, "COPERNICUS/S2_HARMONIZED", yyyymmdd1=yyyymm_start, yyyymmdd2=yyyymm_end, layer_name=f"S2 {yyyymm_start} to {yyyymm_end}")

# Define Sliders 
date_slider = widgets.SelectionSlider(
    options=[str(year) for year in range(2016, 2026)],
    value='2020',
    description='Date:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)

month_slider = widgets.SelectionSlider(
    options=[str(month) for month in range(1, 13)],
    value='1',
    description='Month:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)

time_frame_slider = widgets.SelectionSlider(
    options=[str(month) for month in range(1, 13)],
    value='3',
    description='timeframe:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)


In [ ]:
render_btn = widgets.Button(description="Draw Selected Date Layer", position="bottomright") 
draw_btn = widgets.Button(description="Draw Pins Areas 3yrs", position="bottomright")

widget_draw = widgets.VBox([draw_btn, render_btn])

m.add_widget(widget_draw, position="bottomright")

display(widgets.HBox([date_slider, month_slider, time_frame_slider, render_btn]))
render_btn.on_click(draw_selected)
draw_btn.on_click(draw_basic)

m

# Useful Links 

[Google Earth Engine Editor](https://code.earthengine.google.com/)

[GEE Tasks (Bulk cancel mode)](https://code.earthengine.google.com/tasks)